Copyright (c) Microsoft Corporation. Licensed under the MIT license.

## Recommendation Refresh
---
This script uses the trained ALS model to compute recommendations for targeted users who have new data or signals since the last run of the script.  


In [3]:
from pyspark.ml.recommendation import ALSModel
from pyspark.ml.recommendation import ALS
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.feature import  StringIndexer, IndexToString

In [4]:
model_reload = ALSModel.load("retailai_recommendation_model")

In [30]:
get_top_users = spark.read.table("retailaidb.cleaned_dataset").groupBy(["user_id", "user_session"]).count().groupBy(["user_id"]).count().orderBy("count", ascending=False)

In [33]:
try:
    top_user_ids = spark.read.table("top_users")
except:
    top_user_ids = get_top_users.select("user_id").limit(10)
    top_user_ids.write.saveAsTable("top_users")

In [33]:
preds_alt = model_reload.recommendForUserSubset(top_user_ids, 5).select("user_id", col("recommendations"))

In [30]:
user_preds_df = preds_alt.select("user_id",explode("recommendations").alias("recommendations")) \
                                    .select("user_id", "recommendations.product_id", "recommendations.rating").orderBy("rating", ascending=False)

In [ ]:
final_recommendations = user_preds_df.groupBy("user_id").agg(collect_set("product_id").alias("product_ids"))

In [32]:
final_recommendations.write.format("cosmos.oltp")\
    .option("spark.synapse.linkedService", "retail_ai_cosmos_synapse_link")\
    .option("spark.cosmos.container", "user_recommendations")\
    .option("spark.cosmos.write.upsertEnabled", "true")\
    .mode('append')\
    .save()